In [39]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import display, HTML
import requests
import pandas as pd

In [40]:
proposal_id = '0x1ab7ef84f6e904582d5b5b921944b5b1a8e36dbff1f1248fde87fef02b046816'
endpoint = "https://hub.snapshot.org/graphql"

In [41]:
def fetch_graphql_data(endpoint, query, headers=None):
    response = requests.get(endpoint, json={'query': query})
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [42]:
def get_data(proposal_id):
    choice_query = f"""
    query {{
        proposal(id: "{proposal_id}") {{
            votes
            choices
            scores
        }}
    }}
    """
    
    def get_all_votes(proposal_id):
        all_votes = []
        has_more = True
        skip = 0
        batch_size = 1000

        while has_more:
            results_query = f"""
            query {{
                votes(first: {batch_size}, skip: {skip}, where: {{ proposal: "{proposal_id}" }}) {{
                    voter
                    vp
                    choice
                }}
            }}
            """
            results_data = fetch_graphql_data(endpoint, results_query)
            batch_votes = results_data['data']['votes']
            all_votes.extend(batch_votes)
            
            if len(batch_votes) < batch_size:
                has_more = False
            else:
                skip += batch_size

        return all_votes

    # Get data from snapshot
    choice_data = fetch_graphql_data(endpoint, choice_query)
    results_data = {'votes': get_all_votes(proposal_id)}
    
    return choice_data, results_data

In [43]:
class PairwiseComparison:
    def __init__(self, candidates):
        self.candidates = candidates
        self.candidate_to_index = {candidate: i for i, candidate in enumerate(candidates)}
        self.comparison_matrix = np.zeros((len(candidates), len(candidates)), dtype=int)

    def add_preference(self, preferred, less_preferred, weight):
        i, j = self.candidate_to_index[preferred], self.candidate_to_index[less_preferred]
        self.comparison_matrix[i, j] += weight

    def get_preference_count(self, candidate1, candidate2):
        i, j = self.candidate_to_index[candidate1], self.candidate_to_index[candidate2]
        return self.comparison_matrix[i, j]

    def get_all_pairwise_results(self):
        for i, candidate1 in enumerate(self.candidates):
            for j, candidate2 in enumerate(self.candidates):
                if i < j:
                    yield (candidate1, candidate2, self.comparison_matrix[i, j], self.comparison_matrix[j, i])

    def __len__(self):
        return len(self.candidates) * (len(self.candidates) - 1) // 2

class CopelandElection:
    def __init__(self, candidates, voters, ballots, voter_weights):
        self.candidates = candidates
        self.voters = voters
        
        # List of ballots, where each ballot is a ranked list of candidates
        self.ballots = ballots
        
        # Dictionary mapping each voter to their voting weight (power)
        self.voter_weights = voter_weights

        # Dictionary to store the Copeland score for each candidate
        # Initialized to 0 for each candidate
        self.copeland_scores = {candidate: 0 for candidate in candidates}
        
        self.pairwise_comparison = PairwiseComparison(candidates)
        
        # Dictionary to store the average support for each candidate
        # This is used as a tiebreaker in case of equal Copeland scores
        self.average_support = {candidate: 0 for candidate in candidates}

    def tally_votes(self):
        for ballot, weight in zip(self.ballots, [self.voter_weights[voter] for voter in self.voters]):
            for i, candidate1 in enumerate(ballot):
                for candidate2 in ballot[i+1:]:
                    if candidate1 != "None of the Below" and candidate2 != "None of the Below":
                        self.pairwise_comparison.add_preference(candidate1, candidate2, weight)

        for candidate1, candidate2, votes1, votes2 in self.pairwise_comparison.get_all_pairwise_results():
            if votes1 > votes2:
                self.copeland_scores[candidate1] += 1
            elif votes2 > votes1:
                self.copeland_scores[candidate2] += 1

        total_weight = sum(self.voter_weights.values())
        for candidate in self.candidates:
            self.average_support[candidate] = sum(self.pairwise_comparison.get_preference_count(candidate, other) 
                                                  for other in self.candidates if other != candidate) / (len(self.candidates) - 1)

    def get_winner(self):
        max_score = max(self.copeland_scores.values())
        winners = [c for c, s in self.copeland_scores.items() if s == max_score]
        if len(winners) > 1:
            return max(winners, key=lambda c: self.average_support[c])
        return winners[0]

    def display_results(self):
        # Create a list to store comparison results
        comparisons = []
        
        for candidate1, candidate2, votes1, votes2 in self.pairwise_comparison.get_all_pairwise_results():
            # Extract just the letter from each candidate name
            letter1 = candidate1.split()[-1]
            letter2 = candidate2.split()[-1]
            letter1 = candidate1
            letter2 = candidate2
            
            if letter1 < letter2:  # Ensure consistent ordering
                comparison = f"{letter1} v {letter2}"
                result = f"{votes1} v {votes2}"
                winner = letter1 if votes1 > votes2 else letter2 if votes2 > votes1 else "Tie"
            else:
                comparison = f"{letter2} v {letter1}"
                result = f"{votes2} v {votes1}"
                winner = letter2 if votes2 > votes1 else letter1 if votes1 > votes2 else "Tie"
            
            comparisons.append({
                "Comparison": comparison,
                "Result": result,
                "Winner": winner
            })
        
        # Create DataFrame
        df = pd.DataFrame(comparisons)

    def format_number(self, num):
        if num >= 1000000:
            return f"{num / 1000000:.1f}M"
        elif num >= 10000:
            return f"{num // 1000}k"
        return f"{num:,}"

    def display_results(self):
        # Create a list to store comparison results
        comparisons = []
        
        for candidate1, candidate2, votes1, votes2 in self.pairwise_comparison.get_all_pairwise_results():
            # Extract just the letter from each candidate name
            letter1 = candidate1.split()[-1]
            letter2 = candidate2.split()[-1]
            letter1 = candidate1
            letter2 = candidate2
            
            if letter1 < letter2:  # Ensure consistent ordering
                comparison = f"{letter1} v {letter2}"
                result = f"{self.format_number(votes1)} v {self.format_number(votes2)}"
                winner = letter1 if votes1 > votes2 else letter2 if votes2 > votes1 else "Tie"
            else:
                comparison = f"{letter2} v {letter1}"
                result = f"{self.format_number(votes2)} v {self.format_number(votes1)}"
                winner = letter2 if votes2 > votes1 else letter1 if votes1 > votes2 else "Tie"
            
            comparisons.append({
                "Comparison": comparison,
                "Result": result,
                "Winner": winner
            })
        
        # Create DataFrame
        df = pd.DataFrame(comparisons)

        # Display the table
        styled_df = df.style.set_table_styles([
            {'selector': 'caption', 'props': [('caption-side', 'top'), 
                                              ('font-size', '16px'), 
                                              ('font-weight', 'bold'),
                                              ('color', 'black'),
                                              ('text-align', 'center'),
                                              ('padding', '10px')]},
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'), 
                                         ('color', 'black'),
                                         ('font-weight', 'bold'),
                                         ('text-align', 'center'),
                                         ('border', '1px solid black')]},
            {'selector': 'td', 'props': [('border', '1px solid black'),
                                         ('text-align', 'center')]},
            {'selector': '', 'props': [('border-collapse', 'collapse'),
                                       ('border', '1px solid black')]},

        ]).hide(axis="index")

        # Apply conditional styling for alternating row colors
        styled_df = styled_df.apply(lambda x: ['background-color: #e6f3ff' if x.name % 2 == 0 else '' for _ in x], axis=1)
        styled_df = styled_df.set_caption("Pairwise Results")
        
        # Display the styled table
        display(HTML(styled_df.to_html(index=False)))
        
        #Create rankings table
        rankings = []
        for rank, (candidate, score) in enumerate(sorted(self.copeland_scores.items(), key=lambda x: (-x[1], -self.average_support[x[0]])), 1):
            letter = candidate.split()[-1]
            rankings.append({
                "Rank": rank,
                "Name": letter,
                "Wins": score,
                "Average Support": f"{self.format_number(self.average_support[candidate])}"
            })

        rankings_df = pd.DataFrame(rankings)

        # Style the rankings table
        styled_rankings = rankings_df.style.set_table_styles([
            {'selector': 'caption', 'props': [('caption-side', 'top'), 
                                              ('font-size', '16px'), 
                                              ('font-weight', 'bold'),
                                              ('color', 'black'),
                                              ('text-align', 'center'),
                                              ('padding', '10px')]},
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'), 
                                         ('color', 'black'),
                                         ('font-weight', 'bold'),
                                         ('text-align', 'center'),
                                         ('border', '1px solid black')]},
            {'selector': 'td', 'props': [('border', '1px solid black'),
                                         ('text-align', 'center')]},
            {'selector': '', 'props': [('border-collapse', 'collapse'),
                                       ('border', '1px solid black')]}
        ]).hide(axis="index")

        # Apply conditional styling for alternating row colors
        styled_rankings = styled_rankings.apply(lambda x: ['background-color: #e6f3ff' if x.name % 2 == 0 else '' for _ in x], axis=1)
        styled_rankings = styled_rankings.set_caption("Rankings")

        # Display the rankings table
        display(HTML(styled_rankings.to_html(index=False)))


# Example usage with provided data
candidates = ["Candidate A", "Candidate B", "Candidate C", "Candidate D", "Candidate E","Candidate F"]
voters = ["avsa", "slobo", "frank", "bob", "nick", "rando", "sally", "nancy", "alice"]
voter_weights = {"avsa": 100, "slobo": 10, "frank": 1, "bob": 2, "nick": 3, 
                 "rando": 40, "sally": 50, "nancy": 100000, "alice": 200}

# Example ballots - replace with actual data
ballots = [
    ["Candidate A", "Candidate B", "Candidate C", "None of the Below", "Candidate D", "Candidate E"],
    ["Candidate B", "Candidate C", "Candidate A", "None of the Below", "Candidate E", "Candidate D"],
    ["Candidate C", "Candidate A", "Candidate B", "None of the Below", "Candidate D", "Candidate E"],
    ["Candidate A", "Candidate C", "Candidate B", "None of the Below", "Candidate E", "Candidate D"],
    ["Candidate B", "Candidate A", "Candidate C", "None of the Below", "Candidate D", "Candidate E"],
    ["Candidate C", "Candidate B", "Candidate A", "None of the Below", "Candidate E", "Candidate D"],
    ["Candidate A", "Candidate C", "Candidate B", "None of the Below", "Candidate D", "Candidate E"],
    ["Candidate B", "Candidate A", "Candidate C", "None of the Below", "Candidate E", "Candidate D"],
    ["Candidate C", "Candidate B", "Candidate A", "None of the Below", "Candidate D", "Candidate E"]
]

election = CopelandElection(candidates, voters, ballots, voter_weights)
election.tally_votes()
election.display_results()
print(f"\nNumber of pairwise comparisons: {len(election.pairwise_comparison)}")

Comparison,Result,Winner
Candidate A v Candidate B,153 v 100k,Candidate B
Candidate A v Candidate C,100k v 251,Candidate A
Candidate A v Candidate D,100k v 0,Candidate A
Candidate A v Candidate E,100k v 0,Candidate A
Candidate A v Candidate F,0 v 0,Tie
Candidate B v Candidate C,100k v 293,Candidate B
Candidate B v Candidate D,100k v 0,Candidate B
Candidate B v Candidate E,100k v 0,Candidate B
Candidate B v Candidate F,0 v 0,Tie
Candidate C v Candidate D,100k v 0,Candidate C


Rank,Name,Wins,Average Support
1,B,4,80.0k
2,A,3,60.0k
3,C,2,40.0k
4,E,1,20.0k
5,D,0,70.8
6,F,0,0.0



Number of pairwise comparisons: 15


In [44]:
a,b = get_data(proposal_id)

In [46]:
candidates = a['data']['proposal']['choices']
votes = b['votes']

voters = []
ballots = []
voter_weights = {}

for vote in votes:
    voter = vote['voter']
    voters.append(voter)
    voter_weights[voter] = vote['vp']
    
    # Convert choice indices to candidate names
    ballot = [candidates[i-1] for i in vote['choice']]
    ballots.append(ballot)

# Create and run the election
election = CopelandElection(candidates, voters, ballots, voter_weights)
election.tally_votes()
election.display_results()

# Display additional information
total_votes = a['data']['proposal']['votes']
print(f"\nTotal number of votes: {total_votes}")

Comparison,Result,Winner
Avantgarde v Karpatkey,1.5M v 1.6M,Karpatkey
Avantgarde v Llama,2.1M v 974k,Avantgarde
Avantgarde v None of the above,1.8M v 1.3M,Avantgarde
Karpatkey v Llama,2.2M v 839k,Karpatkey
Karpatkey v None of the above,1.8M v 1.3M,Karpatkey
Llama v None of the above,1.0M v 2.1M,None of the above


Rank,Name,Wins,Average Support
1,Karpatkey,3,1.9M
2,Avantgarde,2,1.8M
3,above,1,1.5M
4,Llama,0,939.0k



Total number of votes: 1496
